# Zeeman beat calculation testing notebook

For estimating Zeeman beat systematics on lifetime measurement. 

## Notes:

- Four level configuration calculation.
- the unit of time, $\Gamma$ is the lifetime of $^3D_1$ state

In [ ]:
# Find and go to the project directory
using DrWatson
@quickactivate "DecayDynamics"
projectname()
cd(projectdir())
pwd()

In [ ]:
# Loading codes
include(srcdir("single_atom_four_level.jl"))
include(srcdir("Base.jl"))

# B field strength check
Check the field strength without any decays. 

In [ ]:
function check_Lamor_precession(Bfield)
	# Parameters for the simulation
	F_i = [1//2, 1//2, 1//2, 1//2] # two intermediate states, 3D1, 3P1, 3P0, 1S0
	g_i = [1, 0.5, 2, 1]*2π
	Γ_kl = sparse(
	        [1, 1, 2],
	        [2, 3, 4],
	        [1*0.385, 1*0.597, 0.1], # Branching ratio, 3P1 assumed to be just 10 times slower. 
	        4, 4
	) * 0
	frac = 1
	tspan = range(0, 20, length=1000)
	ρ_0 = normalize(directsum(
	    (Fm_state(F_i[1], 1//2) + Fm_state(F_i[1], -1//2)) ,
	    (Fm_state(F_i[1], 1//2) + Fm_state(F_i[1], -1//2)),
	    (Fm_state(F_i[1], 1//2) + Fm_state(F_i[1], -1//2)),
	    (Fm_state(F_i[1], 1//2) + Fm_state(F_i[1], -1//2))
	))

	# Pack the parameters
	parameters = @dict F_i g_i Bfield Γ_kl ρ_0 tspan

	# Solve the master equation
	result = evolve_master(parameters)
	figs_population = plot_dynamics(result)
	return plot(figs_population..., size=(1000, 1000), layout=(3, 3), margins=3Plots.mm, suptitle="[Bx, By, Bz] = $(round.(Bfield, digits=3)), g-factor = $(round.(g_i), digits=3)")
end

display(check_Lamor_precession([0, 0, 0.1]))
display(check_Lamor_precession([0, 0.1, 0]))
display(check_Lamor_precession([0.1, 0, 0]))

# Precessions
First check the g-factors:

D state: (1/11, 2/99, -1/9) for F = (11/2, 9/2, 7/2) 

P state: (3/11, 2/33, -1/3) for F = (11/2, 9/2, 7/2) 

The hyperfine splitting for P state is large enough to make the photon distinguishible. Therefore, we neglect the quantum interference effect for that. 

4-level calculation may provide upper bound of the systematics

What would be the initial 3D1 state? Experimental details: 

<img src="../plots/3d1_exp.png" width=500 />

CG coefficients:

<img src="../plots/cg.jpg" width=500 />

Spectroscopy for understanding the spin states.

<img src="../plots/spin_states.png" width=500 />

The initial condition we have is a statistical mixture of all spin states. There is a bit of a bias towards -9/2 states. it is like 10:1:1:...:1. 

From this, we guess that equal superposition of -11/2, -9/2 would give us a upper bound of the Zeeman beat contribution. 


In [ ]:
# population decay and radiation pattern calculation
# compare with zero field or double exponential cases
# save data and fitting with python

## Comparing coherence transfer rates of 3P1 F=11/2 and F=9/2
Which state get more coherence trasnfered?

From the following, we see that the coherence can be more easily transfered to 9/2 manifold, as we can expect from the smaller g-factor difference between upper and lower states.

In [ ]:
"""
	Bfield is in units of Gauss. 76/127 Gauss will give us a Lamor precession with period of Γ
"""
function decay_3P1_11half(Bfield)
	# Parameters for the simulation
	F_i = [11//2, 11//2, 9//2, 9//2] # two intermediate states, 3D1, 3P1, 3P0, 1S0
	g_i = [127, 382, 0, 0]/76 * 2π # 2π from the angular frequency - frequency conversion.
	Γ_kl = sparse(
	        [1, 1, 2],
	        [2, 3, 4],
	        [1*0.385, 1*0.597, 0.1], # Branching ratio, 3P1 assumed to be just 10 times slower. 
	        4, 4
	)
	tspan = range(0, 20, length=3000)
	ρ_0 = normalize(directsum(
		Fm_state(F_i[1], 11//2) + Fm_state(F_i[1], 9//2) ,
		Ket(SpinBasis(F_i[2])),
		Ket(SpinBasis(F_i[3])),
		Ket(SpinBasis(F_i[4]))
	))

	# Pack the parameters
	parameters = @dict F_i g_i Bfield Γ_kl ρ_0 tspan

	# Solve the master equation
	result = evolve_master(parameters)
	figs_population = plot_dynamics(result)
	return plot(figs_population..., size=(1600, 800), layout=(2,4), margins=6Plots.mm, suptitle="[Bx, By, Bz] = $(round.(Bfield, digits=3)), g-factor = $(round.(g_i, digits=3))")
end

function decay_3P1_9half(Bfield)
	# Parameters for the simulation
	F_i = [11//2, 9//2, 9//2, 9//2] # two intermediate states, 3D1, 3P1, 3P0, 1S0
	g_i = [127, 85, 0, 0]/76 * 2π # 2π from the angular frequency - frequency conversion.
	Γ_kl = sparse(
	        [1, 1, 2],
	        [2, 3, 4],
	        [1*0.385, 1*0.597, 0.1], # Branching ratio, 3P1 assumed to be just 10 times slower. 
	        4, 4
	)
	tspan = range(0, 20, length=3000)
	ρ_0 = normalize(directsum(
		Fm_state(F_i[1], 11//2) + Fm_state(F_i[1], 9//2) ,
		Ket(SpinBasis(F_i[2])),
		Ket(SpinBasis(F_i[3])),
		Ket(SpinBasis(F_i[4]))
	))

	# Pack the parameters
	parameters = @dict F_i g_i Bfield Γ_kl ρ_0 tspan

	# Solve the master equation
	result = evolve_master(parameters)
	figs_population = plot_dynamics(result)
	return plot(figs_population..., size=(1600, 800), layout=(2, 4), margins=6Plots.mm, suptitle="[Bx, By, Bz] = $(round.(Bfield, digits=3)), g-factor = $(round.(g_i, digits=3))")
end

display(decay_3P1_11half([0, 0, 76/127]))
display(decay_3P1_9half([0, 0, 76/127]))

## Calculating the radiation


Checking the distance dependence of the radiation pattern

In [ ]:
function scan_distance(rlist)
    # Parameters for the simulation
    F_i = [11//2, 9//2, 9//2, 9//2] # two intermediate states, 3D1, 3P1, 3P0, 1S0
    g_i = [127, 85, 0, 0]/76 * 2π # 2π from the angular frequency - frequency conversion.
    Γ_kl = sparse(
            [1, 1, 2],
            [2, 3, 4],
            [1*0.385, 1*0.597, 0.1], # Branching ratio, 3P1 assumed to be just 10 times slower. 
            4, 4
    )
    tspan = range(0, 77/3, length=300)
    ρ_0 = normalize(directsum(
        Fm_state(F_i[1], 11//2) + Fm_state(F_i[1], 9//2) ,
        Ket(SpinBasis(F_i[2])),
        Ket(SpinBasis(F_i[3])),
        Ket(SpinBasis(F_i[4]))
    ))

    # Pack the parameters
    parameters = @dict F_i g_i Bfield Γ_kl ρ_0 tspan

    # Solve the master equation
    result = evolve_master(parameters)
    # fig_detector = plot_detector_signal(result, pi/2, 0)
    fig = plot()
    mycolor = cgrad(:viridis, length(rlist), categorical=true)
    for ii in eachindex(rlist)
        x, y = get_detector_signal(result, rlist[ii], π/4, 0, NA=0.01, Nsample=1)
        plot!(x, normalize(y), label="r=$(rlist[ii])", color=mycolor[ii])
    end
    fig
end

plot(scan_distance(range(1, 4, length=10)), scan_distance(range(5, 1000, length=10)), size=(1000, 400))


In [ ]:
scan_distance([1e3, 1e4, 1e5, 1e6, 1e10])

r = 1000 or 1000 times the wavelength from the atom seems to be good enough for estimating far field character.

Our imaging system has NA of around 0.2. Check how much random sample for the solid angle is needed to make it converged. 

In [ ]:
function scan_Nsample(Nsamples)
    fig = plot()
    mycolor = cgrad(:viridis, length(Nsamples), categorical=true)
    for ii in eachindex(Nsamples)
        x, y = get_detector_signal(result, 1e3, π/4, 0, NA=0.2, Nsample=Nsamples[ii])
        plot!(x, normalize(y), label="Sample size=$(Nsamples[ii])", color=mycolor[ii])
    end
    fig
end

scan_Nsample([1, 2, 4, 8, 16, 32, 64, ])

Sample size of 30 would be good enough. With this, we finally generate samples with maximal field offset (2 mG) with various angle with respect to the detector or the bias field. 

In [ ]:
"""
    Generate detector signal sample.
    B field set to 2 mG
"""
function generate_sample(θ)
    # Parameters for the simulation
    F_i = [11//2, 9//2, 9//2, 9//2] # two intermediate states, 3D1, 3P1, 3P0, 1S0
    g_i = [127, 85, 0, 0]/76 * 2π # 2π from the angular frequency - frequency conversion.
    Bfield = [0, 0, 2e-3]
    Γ_kl = sparse(
            [1, 1, 2],
            [2, 3, 4],
            [1*0.385, 1*0.597, 0.1], # Branching ratio, 3P1 assumed to be just 10 times slower. 
            4, 4
    )
    tspan = range(0, 77, length=300)
    ρ_0 = normalize(directsum(
        Fm_state(F_i[1], 11//2) + Fm_state(F_i[1], 9//2) ,
        Ket(SpinBasis(F_i[2])),
        Ket(SpinBasis(F_i[3])),
        Ket(SpinBasis(F_i[4]))
    ))

    # Pack the parameters
    parameters = @dict F_i g_i Bfield Γ_kl ρ_0 tspan

    # Solve the master equation
    result = evolve_master(parameters)
    t_out, It = get_detector_signal(result, 1e5, θ, 0, NA=0.2, Nsample=30)
    return t_out, It, result
end

In [ ]:
x, y, result = generate_sample(0)
normalize!(y);

In [ ]:
# plot and fit the data to a double exponential
@. model(x, p) = p[1]*(exp(-(x - p[4])/p[2]) - exp(-(x - p[4])/p[3])) + p[5]
p0 = [maximum(y), 10, 1, 0, 0]
fit_raw = curve_fit(model, x, y, p0)
fit_model = model(x, fit_raw.param)
se = LsqFit.stderror(fit_raw)

fig = plot(x, y, lab="B = 5 mG")
plot!(x, fit_model, lab="Double exponential fit")

fig_res = plot(x, fit_raw.resid, lab="fit residuals")

plot(fig, fig_res, layout=(2, 1), size=(800, 600), suptitle="τD = $(round(fit_raw.param[3], digits=4))($(round(se[3], digits=4))), τP = $(round(fit_raw.param[2], digits=3))($(round(se[2], digits=4)))")

In [ ]:
println(fit_raw.param)
println(se)
